# read obsidian error logs

In [ ]:
import re
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import numpy as np
import os

import error_log_reader
import function_args
import plot_functions

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# Last convergence table, rhat plot, last stats table

In [ ]:
#curr_dir = '/Users/davidkohn/dev/obsidian/output/experiment_03_05_2018_14/'
curr_dir = '/Users/davidkohn/dev/obsidian/output/experiments/05_29_2018/12/'
branch = 'master' # 'asym', 'mhrw', 'adapt'

In [ ]:
if branch == 'asym':
    line_no_converge = 260
    line_no_stats = 250
elif branch == 'mhrw':
    line_no_converge = 252
    line_no_stats = 242
elif branch == 'adapt':
    pass
elif branch == 'master':
    line_no_converge = 257
    line_no_stats = 247
    #line_no_converge = 
    #line_no_stats = 

In [ ]:
# error log stats table
out_stats = error_log_reader.get_info(
    curr_dir,
    **function_args.kwargs_statstable(line_no_stats)
)[0]
array_all = np.stack(out_stats, axis = 2)

df = out_stats[-1]
fpath = os.path.join(curr_dir, 'last_stats_table.csv')
df.to_csv(fpath, index = False)

In [ ]:
# last rhat table
out_convergence = error_log_reader.get_info(
    curr_dir,
    **function_args.kwargs_convergence(line_no_converge)
)[0]

arr = out_convergence[-1]
df = pd.DataFrame(
    [np.array(list(range(len(arr)))), arr]
).T
df.columns = ['ChainID', 'Rhat']
df['ChainID'] = df['ChainID'].astype(int)

fpath = os.path.join(curr_dir, 'last_convergence.csv')
df.to_csv(fpath, index = False)

In [ ]:
# rhat plot
data = out_convergence.T
plot_functions.make_rhat_plot(data, curr_dir)

# Other

# Likelihood search
search for lines with 'likelihood' in them

## gascoyne shard

In [ ]:
out = error_log_reader.get_info(
    dir_gascoyne_01_01,
    **function_args.kwargs_likelihood
)

## plot likelihoods for each parameter type

In [ ]:
fontsize = 20
xstr = 'Log likelihood'
ystr = 'Frequency'
title_str = 'frequency_loglikelihood_{}.png'
fig_height = 10
fig_width = 10

for find_str in find_str_list:
    print(find_str)
    fig = plt.figure(figsize = (fig_width, fig_height))
    for shard in out_likelihoods:
        x = shard[find_str]
        if x:
            plt.hist(x)
    plt.xlabel(
        xstr,
        fontsize = fontsize
    )
    plt.ylabel(
        ystr,
        fontsize = fontsize
    )
    plt.title(
        find_str,
        fontsize = fontsize
    )
    plt.savefig(title_str.format(find_str.replace(' ', '-')))

# Convergence info search
search for lines with convergence info in them

In [ ]:
# recast time strings as datetime objects -> not being used at the moment
"""regex = '([0-9][0-9]:[0-9][0-9]:[0-9][0-9]\.[0-9]+)'
new_time_list = [
    datetime.datetime.strptime((re.findall(regex, time_line)[0]), '%H:%M:%S.%f').time()
    for time_line in time_list
]"""

In [ ]:
out_convergence = error_log_reader.get_info(
    curr_dir,
    **function_args.kwargs_convergence
)[0]

In [ ]:
arr = out_convergence[-1]
df = pd.DataFrame(
    [np.array(list(range(len(arr)))), arr]
).T
df.columns = ['ChainID', 'Rhat']
df['ChainID'] = df['ChainID'].astype(int)

In [ ]:
fpath = os.path.join(curr_dir, 'last_convergence.csv')
df.to_csv(fpath, index = False)

In [ ]:
c = out_convergence.T
param_range = c.shape[0]

fig_width = 10
fig_height = 10
xstr = 'Rhat bin'
ystr = 'Frequency'
title_str = 'Histogram of rhat for all chains'
save_str = 'rhat_histogram'
fontsize = 20

fig = plt.figure(
    figsize = (fig_width, fig_height)
)

ax = fig.gca()
y = c[:, -1]
plt.hist(y)

plt.xlabel(
    xstr,
    fontsize = fontsize
)
plt.ylabel(
    ystr,
    fontsize = fontsize
)
plt.title(
    title_str,
    fontsize = fontsize
)
out_path = os.path.join(
    curr_dir, 
    save_str
)
plt.savefig(
    out_path
)
plt.clf()

In [ ]:
fig_width = 20
fig_height = 20
xstr = 'MCMC iteration'
ystr = 'Rhat'
title_str = 'Rhat over MCMC iterations for all chains'
save_str = 'rhat_iterations'
fontsize = 20
start_slice_idx = 100

fig = plt.figure(
    figsize = (fig_width, fig_height)
)
ax = fig.gca()
for param_idx in range(param_range):
    y = c[param_idx,:]
    y = y[~np.isnan(y)]
    x = range(len(y))
    idx = slice(start_slice_idx, len(y))
    plt.plot(x[idx], y[idx])
    
plt.xlabel(
    xstr,
    fontsize = fontsize
)
plt.ylabel(
    ystr,
    fontsize = fontsize
)
plt.title(
    title_str,
    fontsize = fontsize
)
out_path = os.path.join(
    curr_dir, 
    save_str
)
plt.savefig(
    out_path
)
plt.clf()

In [ ]:
plot_functions.make_convergence_plot(
    c, 
    save_dir = curr_dir,
    param_range = param_range
)

# Stats table search
search for the stats table
the stats table contains the following info:
    0. ChainID
    1. Length - no. samples
    2. MinEngy - lowest energy
    3. CurrEngy - last state
    4. Sigma - proposal width
    5. AcptRt
    6. GlbAcptRt
    7. Beta - inverse temperature of the chain when this state was recorded
        - from obsidian/src/infer/mcmctypes.hpp
    8. SwapRt
    9.GlbWapRt

In [ ]:
out_stats = error_log_reader.get_info(
    curr_dir,
    **function_args.kwargs_statstable
)[0]
array_all = np.stack(out_stats, axis = 2)

In [ ]:
df = out_stats[-1]
fpath = os.path.join(curr_dir, 'last_stats_table.csv')
df.to_csv(fpath, index = False)

In [ ]:
# array_all = np.stack(out_stats, axis = 2)
vals = array_all[3, 8, :]
plt.plot(vals)
plt.show()

# Adaptive testing

In [ ]:
out_stats = error_log_reader.get_info(
    dir_adaptive_test,
    **function_args.kwargs_statstable2
)

In [ ]:
array_all = np.stack(out_stats[0], axis = 2)

In [ ]:
array_all.shape

In [ ]:
out_stats[0][0].columns

In [ ]:
# table 1
array_all[:,:,1]

In [ ]:
vals = array_all[0, 4, :]
plt.plot(vals)
plt.show()

In [ ]:
vals = array_all[1, 4, :]
plt.plot(vals)
plt.show()

In [ ]:
vals = array_all[2, 4, :]
plt.plot(vals)
plt.show()

## cooper basin

In [ ]:
# recast time strings as datetime objects -> unused for now
"""new_time_list = [
    error_log_reader.extract_time_from_string(time_line)
    for time_line in time_list
]"""

# Average evaluation time search

In [ ]:
out_evaltime = error_log_reader.get_info(
    curr_dir,
    **function_args.kwargs_evaluationtime
)

In [ ]:
out_evaltime

In [ ]:
xstr = 'Forward model and likelihood average evaluation time in milliseconds'
ystr = 'Frequency'
title_str = 'Histogram of average evaluation time for sensor {}'
save_str = 'sensor-eval-time-{}.png'
fontsize = 20
fig_width = 10
fig_height = 10
save_dir = curr_dir
for key in out_evaltime[0].keys():
    plot_data = []
    for sub_dict in out_evaltime:
        plot_data += sub_dict[key]
    fig = plt.figure(figsize=(fig_width,fig_height))
    plt.hist(plot_data)
    plt.xlabel(
        xstr,
        fontsize = fontsize
    )
    plt.ylabel(
        ystr,
        fontsize = fontsize
    )
    plt.title(
        title_str.format(key),
        fontsize = fontsize
    )
    out_path = os.path.join(save_dir, save_str.format(key))
    plt.savefig(
        out_path
    )